<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Model-creation" data-toc-modified-id="Model-creation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Model creation</a></span></li><li><span><a href="#Data-Importation" data-toc-modified-id="Data-Importation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Importation</a></span><ul class="toc-item"><li><span><a href="#Text" data-toc-modified-id="Text-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Text</a></span></li><li><span><a href="#Images" data-toc-modified-id="Images-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Images</a></span></li></ul></li><li><span><a href="#Training" data-toc-modified-id="Training-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Training</a></span></li></ul></div>

In [1]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten, Embedding
from keras.layers import Input
from keras.models import Model

from keras.applications import vgg16
from keras.callbacks import ModelCheckpoint
from keras import layers, models, Model, optimizers
from sklearn.model_selection import train_test_split
from keras.layers.core import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import concatenate
import keras
import numpy as np
import argparse
import locale
import datetime
import os

Using TensorFlow backend.


# Model creation

In [6]:
cnn = Sequential()
cnn.add(layers.Flatten(input_shape=(28,28))) # First layer to flatten
cnn.add(layers.Dense(128, activation='relu')) # Second layer with Relu activation
cnn.add(layers.Dropout(0.2)) # Third layer to reduct the dimension
cnn.add(layers.Dense(10, activation='relu')) # We then stack Dense layer
cnn.add(layers.Dense(4, activation='relu')) # And the last one to get the 4 output

emb = Sequential()
emb.add(layers.Dense(10, input_dim=100, activation='relu'))
emb.add(layers.Dense(10, activation='relu'))
emb.add(layers.Dense(10, activation='relu'))
emb.add(layers.Dense(4, activation='relu'))

combinedInput = concatenate([emb.output, 
                             cnn.output])

# We add the final layer (the very last one is up to you, reg or classifier)
x = Dense(4, activation='relu')(combinedInput)

# As said, the last layer is up to you, but here we add a classifier for 7 class
x = Dense(7, activation='softmax')(x)

# And we build the model
model = Model(inputs=[emb.input, cnn.input], outputs=x)

# We define the number of epoch and the validation steps
epochs = 10
validation_steps = 2

# We compile the model
opt = Adam(lr=1e-5, clipnorm = 1.)
loss_fn  = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(loss=loss_fn, optimizer=opt,
              metrics=['accuracy'])

In [7]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
flatten_2_input (InputLayer)    (None, 28, 28)       0                                            
__________________________________________________________________________________________________
dense_13_input (InputLayer)     (None, 100)          0                                            
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 784)          0           flatten_2_input[0][0]            
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 10)           1010        dense_13_input[0][0]             
____________________________________________________________________________________________

# Data Importation

## Text

In [9]:
import pandas as pd

In [13]:
data = pd.read_csv('data/data.csv')
target = pd.read_csv('data/target.csv')

In [14]:
x_train_text = data[data['label'] == 'Train']['description']
x_val_text = data[data['label'] == 'Validation']['description']
x_test_text = data[data['label'] == 'Test']['description']

target.categ = pd.Categorical(target.categ)
target['categ_number'] = np.int32(target.categ.cat.codes)

y_train_text = np.asarray(target[target['label'] == 'Train'][['categ_number']])
y_val_text = np.asarray(target[target['label'] == 'Validation'][['categ_number']])
y_test_text = np.asarray(target[target['label'] == 'Test'][['categ_number']])

In [15]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_train_text)

X_train_text = tokenizer.texts_to_sequences(x_train_text)
X_val_text = tokenizer.texts_to_sequences(x_val_text)
X_test_text = tokenizer.texts_to_sequences(x_test_text)

maxlen = 100
X_train_text = pad_sequences(X_train_text, padding='post', maxlen=maxlen)
X_val_text = pad_sequences(X_val_text, padding='post', maxlen=maxlen)
X_test_text = pad_sequences(X_test_text, padding='post', maxlen=maxlen)

vocab_size = len(tokenizer.word_index) + 1

## Images

In [20]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, load_img

In [21]:
train_data_dir = "data/train"
val_data_dir = "data/validation"
test_data_dir = "data/test"
category_names = sorted(os.listdir('data/train'))
nb_categories = len(category_names)
img_height, img_width = 224,224

In [42]:
#Number of images to load at each iteration
batch_size = 32

# only rescaling
train_datagen =  ImageDataGenerator(
    rescale=1./255
)
test_datagen =  ImageDataGenerator(
    rescale=1./255
)

# these are generators for train/test data that will read pictures 
# found in the defined subfolders of 'data/'
print('Total number of images for "training":')
train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

print('Total number of images for "validation":')
val_generator = test_datagen.flow_from_directory(
val_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical",
shuffle=False)

print('Total number of images for "testing":')
test_generator = test_datagen.flow_from_directory(
test_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical",
shuffle=False)

Total number of images for "training":
Found 630 images belonging to 7 classes.
Total number of images for "validation":
Found 210 images belonging to 7 classes.
Total number of images for "testing":
Found 210 images belonging to 7 classes.


# Training

In [23]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import plot_confusion_matrix

In [63]:
print("[INFO] training model...")
model.fit_generator([X_train_text, train_generator], y_train_text,
                    epochs=10,
                    verbose=1,
                    steps_per_epoch=32
                    #validation_data=([X_val_text, val_generator], y_test_text)
                   )

[INFO] training model...


TypeError: fit_generator() got multiple values for argument 'steps_per_epoch'